From Email:
    
   ```  
Also, we had an issue today.  Trees that were removed from 014Ca Schaller Rd prior to today, were not showing up on the map and not searchable.  When the monitor went to put stump data in today, the trees were gone.  I checked the database to make sure stump data was not entered, and these trees did not have stump status or dates.  
```

### Issue: Trees that are removed but have not had stumps removed are not making it to the toughapads

Issue in the code:
``` 
stump_not_removed_query = _buildNotInWhereClause(
        rem_staging_tbl, 'stump_status', stump_removed)
    query = '"removal_status" <> \'Removed\' AND ' + stump_not_removed_query
    
stump_removed = [
    u'Loam/Seed Complete',
    u'Removed/Treated',
    u'Stump Removed/Treated',
    u'Stump Removed',
    u'Stump Treated',
    u'Stump Ground']
```

Right now, if we run this code, here is what we get

In [1]:
from orl import *
from orl import _buildNotInWhereClause

In [2]:
stump_not_removed_query = _buildNotInWhereClause(
        rem_staging_tbl, 'stump_status', stump_removed)
query = '"removal_status" <> \'Removed\' AND ' + stump_not_removed_query
print query

"removal_status" <> 'Removed' AND stump_status NOT IN('Loam/Seed Complete', 'Removed/Treated', 'Stump Removed/Treated', 'Stump Removed', 'Stump Treated', 'Stump Ground')


In [5]:
# Let's look at how many trees there are on 014Ca Schaller Rd, Bethel, OH 45106

query_addr = "pp_address = '014Ca Schaller Rd, Bethel, OH 45106'"
arcpy.MakeFeatureLayer_management(os.path.join(derived_gdb, 'infested_removals'), 'inf', query_addr)
print arcpy.GetCount_management('inf')
arcpy.Delete_management('inf')

6


<Result 'true'>

As evidenced by the email, and the query, if a tree is removed, but a stump is not removed, this query will not select that record. By changing the AND to and OR, and wrapping the two statement in parentheses, I think we can fix this. 

In [14]:
stump_not_removed_query = _buildNotInWhereClause(
        rem_staging_tbl, 'stump_status', stump_removed)
query2 = 'removal_status <> \'Removed\' OR (' + stump_not_removed_query + ')'
print query2

removal_status <> 'Removed' OR (stump_status NOT IN('Loam/Seed Complete', 'Removed/Treated', 'Stump Removed/Treated', 'Stump Removed', 'Stump Treated', 'Stump Ground'))


In [17]:
def CreateRef_InfestedAndRemovalFC(
        inf_fc, rem_tbl, ref2_gdb, ref_fc, ref_fc_crs,
        ref_fc_fields, fc_join_field_maps, ref_join_fields):
    '''Function that overwrites (or creates) a feature class in
    reference geodatabase of infested trees joined to removals.'''
    arcpy.env.overwriteOutput = True
    if not arcpy.Exists(os.path.join(ref2_gdb, ref_fc)):
        arcpy.CreateFeatureclass_management(ref2_gdb, ref_fc, 'POINT')
        arcpy.DefineProjection_management(
            os.path.join(ref2_gdb, ref_fc), ref_fc_crs)
        for field in ref_fc_fields:
            arcpy.AddField_management(*(os.path.join(
                    ref2_gdb, ref_fc),) + field)
    else:
        arcpy.DeleteRows_management(os.path.join(ref2_gdb, ref_fc))
    # Join Infested and Removals
    stump_not_removed_query = _buildNotInWhereClause(
        rem_staging_tbl, 'stump_status', stump_removed)
    query = '"removal_status" <> \'Removed\' OR (' + stump_not_removed_query + ')'
    arcpy.MakeFeatureLayer_management(inf_fc, 'inf_lyr')
    arcpy.MakeTableView_management(rem_tbl, 'rem_lyr', query)
    arcpy.AddJoin_management(
        'inf_lyr', 'tree_uid', 'rem_lyr', 'tree_uid', 'KEEP_COMMON')
    # print [f.name for f in arcpy.ListFields('inf_fc')]
    # Insert values
    arcpy.MakeFeatureLayer_management(
        os.path.join(ref2_gdb, ref_fc), 'inf_rem_lyr')
    cur = arcpy.da.InsertCursor('inf_rem_lyr', ref_join_fields)
    with arcpy.da.SearchCursor('inf_lyr', fc_join_field_maps) as scur:
        for row in scur:
            cur.insertRow(row)
    # print arcpy.GetCount_management(os.path.join(ref2_gdb, ref_fc))
    logging.info('{} New Removals Layer has {} records'.format(
            dt(), arcpy.GetCount_management(os.path.join(ref2_gdb, ref_fc))))
    return
CreateRef_InfestedAndRemovalFC(
    inf_staging_fc, rem_staging_tbl, derived_gdb, 'infested_removals',
    ohio_crs, Inf_and_Removed_Joined, Inf_and_Removed_Joined_FieldMappings,
    Inf_Joined_FieldMappings)

In [18]:
query_addr = "pp_address = '014Ca Schaller Rd, Bethel, OH 45106'"
arcpy.MakeFeatureLayer_management(os.path.join(derived_gdb, 'infested_removals'), 'inf', query_addr)
print arcpy.GetCount_management('inf')
arcpy.Delete_management('inf')

50


<Result 'true'>

# Fix

* Replace the original query using AND with OR and parenthesis, as in the function above.